# Step 1: Install YOLOv8 Dependencies
First, we'll install the necessary dependencies, such as ultralytics, which includes the YOLOv8 implementation.

In [1]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 19.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 79.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 81.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 87.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 75.4 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 87.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 85.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 751.2/751.2 kB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.7/37.7 MB 77.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 888.0/888.0 MB 56.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 69.0 MB/s eta 0:00:0000:0100:01
   ━━━

In [3]:
# 필요한 라이브러리 import
import glob  # 파일 목록 탐색용
import os    # 파일 경로 및 존재 확인용

def calculate_f1_score(ground_truth_dir, pred_dir):
    true_positives = 0
    false_positives = 0
    false_negatives = 0
    gt_files = glob.glob(f'{ground_truth_dir}/*.txt')
    for gt_file in gt_files:
        img_name = os.path.splitext(os.path.basename(gt_file))[0]
        pred_file = os.path.join(pred_dir, f'{img_name}.txt')
        gt_boxes = []
        if os.path.exists(gt_file):
            with open(gt_file, 'r') as f:
                for line in f:
                    parts = list(map(float, line.strip().split()))
                    gt_boxes.append((int(parts[0]), parts[1:]))
        pred_boxes = []
        if os.path.exists(pred_file):
            with open(pred_file, 'r') as f:
                for line in f:
                    parts = list(map(float, line.strip().split()))
                    pred_boxes.append((int(parts[0]), parts[1:5], parts[5]))
        matched_gt = set()
        for pred_class, pred_box, conf in pred_boxes:
            matched = False
            for i, (gt_class, gt_box) in enumerate(gt_boxes):
                if pred_class == gt_class and i not in matched_gt:
                    iou = calculate_iou(pred_box, gt_box)
                    if iou > 0.45:
                        true_positives += 1
                        matched_gt.add(i)
                        matched = True
                        break
            if not matched:
                false_positives += 1
        false_negatives += len(gt_boxes) - len(matched_gt)
    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    return precision, recall, f1_score

def calculate_iou(box1, box2):
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2
    x1_min, y1_min = x1 - w1/2, y1 - h1/2
    x1_max, y1_max = x1 + w1/2, y1 + h1/2
    x2_min, y2_min = x2 - w2/2, y2 - h2/2
    x2_max, y2_max = x2 + w2/2, y2 + h2/2
    inter_x_min = max(x1_min, x2_min)
    inter_y_min = max(y1_min, y2_min)
    inter_x_max = min(x1_max, x2_max)
    inter_y_max = min(y1_max, y2_max)
    inter_area = max(0, inter_x_max - inter_x_min) * max(0, inter_y_max - inter_y_min)
    box1_area = w1 * h1
    box2_area = w2 * h2
    union_area = box1_area + box2_area - inter_area
    return inter_area / union_area if union_area > 0 else 0

# Step 2: import & Load a Pre-trained YOLOv8 Model
YOLOv8 comes with pre-trained models that can be used directly for inference or fine-tuned for your specific dataset. You can load the pre-trained model like this:

In [4]:
from ultralytics import YOLO

WARNING ⚠️ user config directory '/home/heodnjswns/.config/Ultralytics' is not writeable, defaulting to '/tmp' or CWD. Alternatively you can define a YOLO_CONFIG_DIR environment variable for this path.
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/tmp/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [6]:
# Load a YOLOv8 model

# model = YOLO('yolov8x.pt')
model = YOLO("yolov8n.pt")

In [ ]:
# !pip install roboflow

In [ ]:
# from roboflow import Roboflow
# rf = Roboflow(api_key="zVwIE1yVxhpq2eF9mjtx")
# project = rf.workspace("bccdyolodataset").project("road_yolo-epmit")
# version = project.version(1)
# dataset = version.download("yolov8")

You have 5 model sizes:

1.   XL
2.   Large
3.  Medium
4.  Small
5.  Tiny
**You can choose between yolov8n.pt, yolov8s.pt, yolov8m.pt, yolov8l.pt, yolov8x.pt**


yolov8n.pt is the "nano" version, which is smaller and faster but less accurate.

yolov8x.pt is the "extra-large" version, which has the highest accuracy but requires more computational resources.

# Step 3: Prepare Your Dataset
If you want to train a custom object detection model, you’ll need to have a dataset in the following format:

Images: Stored in a folder (e.g., /images/).

Labels: Text files for each image with bounding box coordinates in the format [class_id, x_center, y_center, width, height], normalized between 0 and 1 (e.g., /labels/).


For training, you'll also need to create a data.yaml file with paths to your train and validation sets, as well as the class labels.


```
train: /path/to/train/images
val: /path/to/val/images

nc: 3  # Number of classes
names: ['class1', 'class2', 'class3']  # Class names

```





# YAML File for YOLO
To train a YOLO model using the costom dataset, you need to create a YAML file that defines the dataset's classes. Here's an example of how the YAML file should look:


```
train: /home/work/wonjun/usc/yolov8/ROAD_YOLO-1/train/images
val: /home/work/wonjun/usc/yolov8/ROAD_YOLO-1/val
test: /home/work/wonjun/usc/yolov8/ROAD_YOLO-1/test
nc: 4
names:
  0: 'Pothole'
  1: 'Alligator Crack'
  2: 'Transverse Crack'
  3: 'Longitudinal Crack'
```
In this YAML file:

* train, test, and val: Specify the directories containing your training and validation images, respectively.

* nc: Indicates the number of classes (4 in this case).

* names: Lists the class names corresponding to the class IDs.




# Step 4: Train YOLOv8 Model
To train your custom YOLOv8 model, use the following command. Here we assume you've uploaded your dataset and have a data.yaml file.


In [ ]:
# Train the YOLOv8 model on your custom dataset
model = YOLO('yolov8n.pt')  # Load pre-trained model
model.train(data='/home/heodnjswns/cracktracker/yolo_proj/data.yaml', epochs=30, imgsz=640, batch=32, lr0=0.001, lrf=0.1, optimizer='AdamW')

CUDA_VISIBLE_DEVICES = 5
torch sees 6 GPU(s)
Using: NVIDIA RTX 6000 Ada Generation
Ultralytics 8.3.177 🚀 Python-3.10.18 torch-2.8.0+cu128 CUDA:3 (NVIDIA RTX 6000 Ada Generation, 48520MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/home/heodnjswns/cracktracker/yolo_proj/data.yaml, degrees=0.0, deterministic=True, device=3, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.1, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train8, nbs=64, nms=False, ops

train: Scanning /home/heodnjswns/cracktracker/yolo_proj/dataset/train/labels.cache... 7058 images, 5 backgrounds, 0 corrupt: 100%|██████████| 7058/7058 [00:00<?, ?it/s]


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 707.6±103.3 MB/s, size: 39.0 KB)


val: Scanning /home/heodnjswns/cracktracker/yolo_proj/dataset/valid/labels.cache... 2048 images, 3 backgrounds, 0 corrupt: 100%|██████████| 2048/2048 [00:00<?, ?it/s]


Plotting labels to runs/detect/train8/labels.jpg... 
optimizer: AdamW(lr=0.001, momentum=0.937) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/detect/train8
Starting training for 30 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/30        10G      1.278      1.172      1.154         20        640: 100%|██████████| 221/221 [00:19<00:00, 11.17it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  75%|███████▌  | 24/32 [00:03<00:01,  7.27it/s]

# Step 5: Evaluate Your Model
After training, you can evaluate the model's performance using the validation dataset:

In [ ]:
# Evaluate the trained model
results = model.val()
print(results)

Ultralytics 8.3.171 🚀 Python-3.10.18 torch-2.7.1+cu126 CUDA:0 (NVIDIA H100 80GB HBM3, 24468MiB)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access ✅ (ping: 0.1±0.0 ms, read: 267.7±54.9 MB/s, size: 41.6 KB)


val: Scanning /home/work/wonjun/usc/yolo_proj/dataset/valid/labels.cache... 2048 images, 3 backgrounds, 0 corrupt: 100%|██████████| 2048/2048 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 256/256 [00:12<00:00, 20.60it/s]


                   all       2048       2134      0.969      0.908      0.955       0.67
Speed: 0.1ms preprocess, 1.1ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to runs/detect/train62
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f1fb043fc70>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.03

In [ ]:
result = model.predict(source='/home/work/wonjun/usc/yolo_proj/dataset/test/images', conf=0.25, save=True, save_txt=True, save_conf=True)

# Step 6: Test the Model
To test the model on new, unseen images, you can use the following code:


In [ ]:
# model = YOLO('/home/work/wonjun/usc/yolov8/runs/detect/train251404_base/weights/best.pt')

In [ ]:
# Run inference on new images
# results = model.predict(source='/home/work/wonjun/usc/yolov8/dataset/test/images', save=True)

# results

# Step 7: Save the Model
To save the trained model for later use, use the following command:

In [ ]:
# Save the model weights
# model.save('yolov8_251404.pt')

# 결과 내보기

In [11]:
# Import necessary libraries
from ultralytics import YOLO
import os
import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Load the pre-trained model

model_path = '/home/work/wonjun/usc/yolo_proj/yolov11n_basedataset_736(30).pt'
model = YOLO(model_path)# model_path = '/home/work/wonjun/usc/yolov8/runs/detect/train2/weights/best260151_v11_xl_base.pt'  # 최신 학습된 모델 경로

# Extract model name from file path and create subfolder
model_name = os.path.splitext(os.path.basename(model_path))[0]
sub_dir = os.path.join('predicted_results', model_name)  # 서브디렉토리
os.makedirs(sub_dir, exist_ok=True)  # 서브디렉토리 생성 (존재하면 무시)


print(f"Libraries imported and model loaded successfully.")

Libraries imported and model loaded successfully.


In [12]:
# Function to process and save predictions in required format
# - Saves predictions as 'class x y w h confidence' per line
# - Visualizes predictions for verification
def save_predictions(model, image_dir, output_dir):
    for img_path in glob.glob(f'{image_dir}/*.jpg'):
        results = model.predict(img_path, conf=0.25)  # Confidence threshold 0.25 for balanced detection
        img_name = os.path.splitext(os.path.basename(img_path))[0]
        output_file = os.path.join(output_dir, f'{img_name}.txt')

        with open(output_file, 'w') as f:
            for result in results:
                boxes = result.boxes.xywhn  # Normalized x, y, w, h
                confidences = result.boxes.conf
                classes = result.boxes.cls
                for box, conf, cls in zip(boxes, confidences, classes):
                    x, y, w, h = box
                    class_id = int(cls.item())
                    confidence = float(conf.item())
                    f.write(f"{class_id} {x:.6f} {y:.6f} {w:.6f} {h:.6f} {confidence:.6f}\n")

        # Visualize predictions
        # annotated_img = result.plot()
        # plt.imshow(cv2.cvtColor(annotated_img, cv2.COLOR_BGR2RGB))
        # plt.title(f'Predictions for {img_name}')
        # plt.axis('off')
        # plt.savefig(os.path.join(output_dir, f'{img_name}.jpg'))
        # plt.close()

print("Prediction function defined.")

Prediction function defined.


In [13]:
# Set image directory and run prediction

test_image_dir = '/home/work/wonjun/usc/yolo_proj/dataset/test/images'    # 고정
save_predictions(model, test_image_dir, sub_dir)

print(f"Predictions saved to {sub_dir}")


image 1/1 /home/work/wonjun/usc/yolo_proj/dataset/test/images/0010f4c10f7ab07e_jpg.rf.92344aa620e23aacc490273e32343595.jpg: 736x736 1 License_Plate, 7.2ms
Speed: 3.5ms preprocess, 7.2ms inference, 4.4ms postprocess per image at shape (1, 3, 736, 736)

image 1/1 /home/work/wonjun/usc/yolo_proj/dataset/test/images/000812dcf304a8e7_jpg.rf.559f904bc045f68ee947796a1b561d8f.jpg: 736x736 1 License_Plate, 7.3ms
Speed: 2.9ms preprocess, 7.3ms inference, 2.7ms postprocess per image at shape (1, 3, 736, 736)



image 1/1 /home/work/wonjun/usc/yolo_proj/dataset/test/images/0002a5b67e5f0909_jpg.rf.07ca41e79eb878b14032f650f34d0967.jpg: 736x736 2 License_Plates, 6.9ms
Speed: 2.3ms preprocess, 6.9ms inference, 3.3ms postprocess per image at shape (1, 3, 736, 736)

image 1/1 /home/work/wonjun/usc/yolo_proj/dataset/test/images/0026c246d5c33bea_jpg.rf.782def291f7881f554c7a8bd17b9ff13.jpg: 736x736 3 License_Plates, 9.0ms
Speed: 2.8ms preprocess, 9.0ms inference, 6.7ms postprocess per image at shape (1, 3, 736, 736)

image 1/1 /home/work/wonjun/usc/yolo_proj/dataset/test/images/001cdd25e148cd36_jpg.rf.eca53e55e3cf5986bcfd95bc277b6438.jpg: 736x736 1 License_Plate, 7.3ms
Speed: 2.3ms preprocess, 7.3ms inference, 3.4ms postprocess per image at shape (1, 3, 736, 736)

image 1/1 /home/work/wonjun/usc/yolo_proj/dataset/test/images/002901d9d194c4fb_jpg.rf.292313a88fa94fe76a51739acc95b5ab.jpg: 736x736 1 License_Plate, 8.1ms
Speed: 2.2ms preprocess, 8.1ms inference, 15.7ms postprocess per image at shape (1, 3, 

In [14]:
# Calculate F1 score
ground_truth_dir = '/home/work/wonjun/usc/yolo_proj/dataset/test/labels'  # Ground truth labels directory
# precision, recall, f1_score = calculate_f1_score(ground_truth_dir, sub_dir)
precision, recall, f1_score = calculate_f1_score(ground_truth_dir, sub_dir)

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1_score:.4f}")

Precision: 0.9717
Recall: 0.9809
F1 Score: 0.9763


# 예측 시 증강

In [15]:
from torchvision.ops import nms
import torch

def save_predictions_with_aug(model, image_dir, output_dir, conf_thres=0.4, iou_thres=0.45):
    os.makedirs(output_dir, exist_ok=True)
    for img_path in glob.glob(f'{image_dir}/*.jpg'):
        img = cv2.imread(img_path)
        contrast_img = cv2.convertScaleAbs(img, alpha=1.8, beta=0)
        kernel = np.array([[0, -1, 0], [-1, 5,-1], [0, -1, 0]])
        sharpen_img = cv2.filter2D(img, -1, kernel)

        aug_imgs = [contrast_img, sharpen_img]
        all_boxes = []
        all_scores = []
        all_classes = []

        for aug_img in aug_imgs:
            temp_path = 'temp_aug.jpg'
            cv2.imwrite(temp_path, aug_img)
            results = model.predict(temp_path, conf=conf_thres)
            for result in results:
                boxes = result.boxes.xywhn
                confidences = result.boxes.conf
                classes = result.boxes.cls
                for box, conf, cls in zip(boxes, confidences, classes):
                    all_boxes.append(box.cpu().numpy())
                    all_scores.append(float(conf.item()))
                    all_classes.append(int(cls.item()))
            os.remove(temp_path)

        # NMS 적용 (xywhn -> xyxy 변환 필요)
        if all_boxes:
            all_boxes = np.array(all_boxes)
            # xywhn -> xyxy 변환
            xyxy_boxes = []
            for box in all_boxes:
                x, y, w, h = box
                x1 = x - w / 2
                y1 = y - h / 2
                x2 = x + w / 2
                y2 = y + h / 2
                xyxy_boxes.append([x1, y1, x2, y2])
            boxes_tensor = torch.tensor(xyxy_boxes, dtype=torch.float32)
            scores_tensor = torch.tensor(all_scores, dtype=torch.float32)
            classes_tensor = torch.tensor(all_classes, dtype=torch.int64)

            keep = nms(boxes_tensor, scores_tensor, iou_thres)
            output_file = os.path.join(output_dir, f'{os.path.splitext(os.path.basename(img_path))[0]}.txt')
            with open(output_file, 'w') as f:
                for idx in keep:
                    # 다시 xywhn로 변환
                    x1, y1, x2, y2 = boxes_tensor[idx]
                    x = (x1 + x2) / 2
                    y = (y1 + y2) / 2
                    w = x2 - x1
                    h = y2 - y1
                    class_id = int(classes_tensor[idx].item())
                    confidence = float(scores_tensor[idx].item())
                    f.write(f"{class_id} {x:.6f} {y:.6f} {w:.6f} {h:.6f} {confidence:.6f}\n")
    print("Augmented predictions saved with NMS.")

# 사용 예시
model_path = '/home/work/wonjun/usc/yolo_proj/yolov11n_basedataset_640(30).pt'
model = YOLO(model_path)
output_dir = os.path.join('predicted_results', os.path.splitext(os.path.basename(model_path))[0], 'aug_only')
test_image_dir = '/home/work/wonjun/usc/yolo_proj/dataset/test/images'
save_predictions_with_aug(model, test_image_dir, output_dir)

ground_truth_dir = '/home/work/wonjun/usc/yolo_proj/dataset/test/labels' 
precision, recall, f1_score = calculate_f1_score(ground_truth_dir, output_dir)

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1_score:.4f}")


image 1/1 /home/work/wonjun/usc/yolo_proj/temp_aug.jpg: 640x640 1 License_Plate, 7.0ms
Speed: 2.3ms preprocess, 7.0ms inference, 4.0ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /home/work/wonjun/usc/yolo_proj/temp_aug.jpg: 640x640 1 License_Plate, 6.9ms
Speed: 2.0ms preprocess, 6.9ms inference, 3.8ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /home/work/wonjun/usc/yolo_proj/temp_aug.jpg: 640x640 1 License_Plate, 10.9ms
Speed: 1.4ms preprocess, 10.9ms inference, 28.1ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /home/work/wonjun/usc/yolo_proj/temp_aug.jpg: 640x640 1 License_Plate, 7.3ms
Speed: 1.2ms preprocess, 7.3ms inference, 15.0ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /home/work/wonjun/usc/yolo_proj/temp_aug.jpg: 640x640 1 License_Plate, 7.6ms
Speed: 1.6ms preprocess, 7.6ms inference, 7.2ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /home/work/wonjun/usc/yolo_proj/temp_aug.jpg: 640x640 1 License_P

Precision: 0.9753
Recall: 0.9781
F1 Score: 0.9767


Augmented Predictions F1 Score: 0.9767


In [10]:
import os
import glob
import cv2
import numpy as np
from ultralytics import YOLO

def preprocess_image_multi(img):
    # 대비 증가
    img = cv2.convertScaleAbs(img, alpha=1.8, beta=0)
    # 샤프닝
    kernel = np.array([[0, -1, 0], [-1, 5,-1], [0, -1, 0]])
    img = cv2.filter2D(img, -1, kernel)
    return img

def preprocess_variants(img):
    variants = []
    # 원본
    variants.append(img.copy())
    # 대비
    variants.append(cv2.convertScaleAbs(img, alpha=1.8, beta=0))
    # 샤프닝
    kernel = np.array([[0, -1, 0], [-1, 5,-1], [0, -1, 0]])
    variants.append(cv2.filter2D(img, -1, kernel))
    return variants

def save_predictions_tta_mean(model, image_dir, output_dir):
    os.makedirs(os.path.join(output_dir, 'tta_mean'), exist_ok=True)
    for img_path in glob.glob(f'{image_dir}/*.jpg'):
        img = cv2.imread(img_path)
        aug_imgs = preprocess_variants(img)
        all_boxes = []
        all_confs = []
        all_classes = []
        for aug_img in aug_imgs:
            temp_path = 'temp_tta.jpg'
            cv2.imwrite(temp_path, aug_img)
            results = model.predict(temp_path, conf=0.25)
            for result in results:
                boxes = result.boxes.xywhn
                confidences = result.boxes.conf
                classes = result.boxes.cls
                for box, conf, cls in zip(boxes, confidences, classes):
                    all_boxes.append(box.cpu().numpy())
                    all_confs.append(float(conf.item()))
                    all_classes.append(int(cls.item()))
            os.remove(temp_path)
        # 평균 결과 산출 (클래스별로 평균)
        if all_boxes:
            avg_boxes = np.mean(np.array(all_boxes), axis=0)
            avg_conf = np.mean(all_confs)
            avg_class = max(set(all_classes), key=all_classes.count)  # 가장 많이 나온 클래스
            img_name = os.path.splitext(os.path.basename(img_path))[0]
            output_file = os.path.join(output_dir, 'tta_mean', f'{img_name}.txt')
            with open(output_file, 'w') as f:
                # 여러 박스가 있을 경우 모두 저장
                f.write(f"{avg_class} {avg_boxes[0]:.6f} {avg_boxes[1]:.6f} {avg_boxes[2]:.6f} {avg_boxes[3]:.6f} {avg_conf:.6f}\n")
    print("tta_mean prediction saved.")

# 모델 및 경로 설정
model_path = '/home/work/wonjun/usc/yolo_proj/yolov11x_720_b32(30).pt'
model = YOLO(model_path)
sub_dir = os.path.join('predicted_results', os.path.splitext(os.path.basename(model_path))[0])
os.makedirs(sub_dir, exist_ok=True)
test_image_dir = '/home/work/wonjun/usc/yolo_proj/dataset/test/images'

# TTA 평균 방식으로 예측 저장
save_predictions_tta_mean(model, test_image_dir, sub_dir)

# 성능 평가
ground_truth_dir = '/home/work/wonjun/usc/yolo_proj/dataset/test/labels'
tta_pred_dir = os.path.join(sub_dir, 'tta_mean')
precision, recall, f1_score = calculate_f1_score(ground_truth_dir, tta_pred_dir)
print(f"TTA Mean Precision: {precision:.4f}")
print(f"TTA Mean Recall: {recall:.4f}")
print(f"TTA Mean F1 Score: {f1_score:.4f}")


image 1/1 /home/work/wonjun/usc/yolo_proj/temp_tta.jpg: 736x736 2 License_Plates, 14.7ms
Speed: 2.4ms preprocess, 14.7ms inference, 2.4ms postprocess per image at shape (1, 3, 736, 736)

image 1/1 /home/work/wonjun/usc/yolo_proj/temp_tta.jpg: 736x736 1 License_Plate, 15.5ms
Speed: 2.3ms preprocess, 15.5ms inference, 5.4ms postprocess per image at shape (1, 3, 736, 736)

image 1/1 /home/work/wonjun/usc/yolo_proj/temp_tta.jpg: 736x736 1 License_Plate, 15.3ms
Speed: 2.3ms preprocess, 15.3ms inference, 5.7ms postprocess per image at shape (1, 3, 736, 736)

image 1/1 /home/work/wonjun/usc/yolo_proj/temp_tta.jpg: 736x736 1 License_Plate, 18.6ms
Speed: 2.4ms preprocess, 18.6ms inference, 14.6ms postprocess per image at shape (1, 3, 736, 736)

image 1/1 /home/work/wonjun/usc/yolo_proj/temp_tta.jpg: 736x736 1 License_Plate, 15.2ms
Speed: 2.5ms preprocess, 15.2ms inference, 7.4ms postprocess per image at shape (1, 3, 736, 736)

image 1/1 /home/work/wonjun/usc/yolo_proj/temp_tta.jpg: 736x736 1 L

# 앙상블

In [21]:
import torch
import cv2
import os
import glob
import matplotlib.pyplot as plt
from ultralytics import YOLO
from torchvision.ops import nms

# 클래스 이름 정의 (기존 YAML 파일 기반)
class_names = ['License_Plate']

# 디렉토리 및 경로 정의
# test_image_dir = '/home/work/wonjun/usc/yolov8/dataset/test/images'  # 테스트 이미지 디렉토리
test_image_dir = '/home/work/wonjun/usc/yolo_proj/dataset/test/images'  # 테스트 이미지 디렉토리
ground_truth_dir = '/home/work/wonjun/usc/yolo_proj/dataset/test/labels'  # Ground truth 레이블 디렉토리
output_dir = 'predicted_results/ensemble'  # 예측 결과 저장 디렉토리
os.makedirs(output_dir, exist_ok=True)  # 출력 디렉토리 생성

# 모델 로드
model1 = YOLO('/home/work/wonjun/usc/yolo_proj/yolov11n_basedataset_640(30).pt')

model2 = YOLO('/home/work/wonjun/usc/yolo_proj/yolov11x_720_b32(30).pt')

# 예측 결과 추출 함수
def extract_predictions(results):
    try:
        boxes = results.boxes.xyxy  # (x1, y1, x2, y2)
        scores = results.boxes.conf
        classes = results.boxes.cls
        if boxes is None or len(boxes) == 0:
            return torch.tensor([]).to(torch.float32), torch.tensor([]).to(torch.float32), torch.tensor([]).to(torch.int64)
        return boxes, scores, classes
    except Exception as e:
        print(f"Error extracting predictions: {e}")
        return torch.tensor([]).to(torch.float32), torch.tensor([]).to(torch.float32), torch.tensor([]).to(torch.int64)

# 앙상블 예측 저장 함수 (YOLO 형식: class_id x_center y_center width height confidence)
def save_ensemble_predictions(boxes, scores, classes, output_dir, img_path):
    img_name = os.path.splitext(os.path.basename(img_path))[0]
    output_file = os.path.join(output_dir, f"{img_name}.txt")
    
    with open(output_file, 'w') as f:
        if len(boxes) > 0:
            img = cv2.imread(img_path)
            h, w = img.shape[:2]
            for box, score, cls in zip(boxes, scores, classes):
                x1, y1, x2, y2 = box.tolist()
                x_center = (x1 + x2) / 2 / w
                y_center = (y1 + y2) / 2 / h
                width = (x2 - x1) / w
                height = (y2 - y1) / h
                class_id = int(cls.item())
                confidence = float(score.item())
                f.write(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f} {confidence:.6f}\n")

# 예측 시각화 함수
def visualize_predictions(img_path, boxes, scores, classes, class_names, output_dir):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_name = os.path.splitext(os.path.basename(img_path))[0]
    
    for box, score, cls in zip(boxes, scores, classes):
        x1, y1, x2, y2 = box.int().tolist()
        class_id = int(cls.item())
        label = f"{class_names[class_id]}: {score:.2f}"
        color = (0, 255, 0)  # Green
        cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
        cv2.putText(img, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    
    plt.imshow(img)
    plt.axis('off')
    plt.title(f"Ensemble Predictions: {img_name}")
    plt.savefig(os.path.join(output_dir, f"{img_name}.jpg"))
    plt.close()

# 앙상블 예측 함수
def ensemble_predictions(model1, model2, image_dir, output_dir):
    for img_path in glob.glob(f'{image_dir}/*.jpg'):
        try:
            # 두 모델로 예측
            results1 = model1.predict(img_path, conf=0.25)[0]
            results2 = model2.predict(img_path, conf=0.25)[0]
            
            # 예측 결과 추출
            boxes1, scores1, classes1 = extract_predictions(results1)
            boxes2, scores2, classes2 = extract_predictions(results2)
            
            # 예측 결합
            all_boxes = torch.cat([boxes1, boxes2], dim=0)
            all_scores = torch.cat([scores1, scores2], dim=0)
            all_classes = torch.cat([classes1, classes2], dim=0)
            
            # 클래스별 NMS
            final_boxes, final_scores, final_classes = [], [], []
            for cls in torch.unique(all_classes):
                cls_mask = all_classes == cls
                cls_boxes = all_boxes[cls_mask]
                cls_scores = all_scores[cls_mask]
                indices = nms(cls_boxes, cls_scores, iou_threshold=0.60)
                final_boxes.append(cls_boxes[indices])
                final_scores.append(cls_scores[indices])
                final_classes.append(all_classes[cls_mask][indices])
            
            # 최종 결과 결합
            final_boxes = torch.cat(final_boxes) if final_boxes else torch.tensor([]).to(torch.float32)
            final_scores = torch.cat(final_scores) if final_scores else torch.tensor([]).to(torch.float32)
            final_classes = torch.cat(final_classes) if final_classes else torch.tensor([]).to(torch.int64)
            
            # 결과 저장 및 시각화
            save_ensemble_predictions(final_boxes, final_scores, final_classes, output_dir, img_path)
            # visualize_predictions(img_path, final_boxes, final_scores, final_classes, class_names, output_dir)
            
            print(f"Processed: {os.path.basename(img_path)}")
        except Exception as e:
            print(f"Error processing {img_path}: {e}")

# 앙상블 예측 실행
print("Starting ensemble predictions...")
ensemble_predictions(model1, model2, test_image_dir, output_dir)
print(f"Predictions saved to {output_dir}")

# F1 스코어 계산
precision, recall, f1_score = calculate_f1_score(ground_truth_dir, output_dir)
print(f"Ensemble Precision: {precision:.4f}")
print(f"Ensemble Recall: {recall:.4f}")
print(f"Ensemble F1 Score: {f1_score:.4f}")

Starting ensemble predictions...

image 1/1 /home/work/wonjun/usc/yolo_proj/dataset/test/images/0010f4c10f7ab07e_jpg.rf.92344aa620e23aacc490273e32343595.jpg: 640x640 1 License_Plate, 6.4ms
Speed: 1.2ms preprocess, 6.4ms inference, 2.2ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /home/work/wonjun/usc/yolo_proj/dataset/test/images/0010f4c10f7ab07e_jpg.rf.92344aa620e23aacc490273e32343595.jpg: 736x736 2 License_Plates, 15.4ms
Speed: 2.2ms preprocess, 15.4ms inference, 1.4ms postprocess per image at shape (1, 3, 736, 736)
Processed: 0010f4c10f7ab07e_jpg.rf.92344aa620e23aacc490273e32343595.jpg

image 1/1 /home/work/wonjun/usc/yolo_proj/dataset/test/images/000812dcf304a8e7_jpg.rf.559f904bc045f68ee947796a1b561d8f.jpg: 640x640 1 License_Plate, 7.6ms
Speed: 1.1ms preprocess, 7.6ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /home/work/wonjun/usc/yolo_proj/dataset/test/images/000812dcf304a8e7_jpg.rf.559f904bc045f68ee947796a1b561d8f.jpg: 736x736 1 L

# 3개 앙상블

In [ ]:
import torch
import cv2
import os
import glob
import matplotlib.pyplot as plt
from ultralytics import YOLO
from torchvision.ops import nms
from itertools import combinations

# 클래스 이름 정의 (기존 YAML 파일 기반)
class_names = ['Pothole', 'Alligator Crack', 'Transverse Crack', 'Longitudinal Crack']

# 디렉토리 및 경로 정의
test_image_dir = '/home/work/wonjun/usc/yolov8/dataset/test/images'  # 테스트 이미지 디렉토리
ground_truth_dir = '/home/work/wonjun/usc/yolov8/dataset/test/labels'  # Ground truth 레이블 디렉토리
base_output_dir = 'predicted_results'  # 기본 출력 디렉토리
os.makedirs(base_output_dir, exist_ok=True)

# 모델 로드
model1 = YOLO('/home/work/wonjun/usc/yolov8/runs/detect/train251404_v8_xl_base/weights/best_yolov8x_base.pt')
model2 = YOLO('/home/work/wonjun/usc/yolov8/runs/detect/train260151_v11_l_base/weights/best260151_v11_l_base.pt')
model3 = YOLO('/home/work/wonjun/usc/yolov8/runs/detect/train260151_v11_xl_base/weights/best260151_v11_xl_base.pt')
all_models = [model1, model2, model3]  # 모든 모델 리스트
model_names = ['YOLOv8xl', 'YOLOv11l', 'YOLOv11xl']  # 모델 이름
print("Models loaded successfully:", model_names)

# 예측 결과 추출 함수
def extract_predictions(results):
    try:
        boxes = results.boxes.xyxy  # (x1, y1, x2, y2)
        scores = results.boxes.conf
        classes = results.boxes.cls
        if boxes is None or len(boxes) == 0:
            return torch.tensor([]).to(torch.float32), torch.tensor([]).to(torch.float32), torch.tensor([]).to(torch.int64)
        return boxes, scores, classes
    except Exception as e:
        print(f"Error extracting predictions: {e}")
        return torch.tensor([]).to(torch.float32), torch.tensor([]).to(torch.float32), torch.tensor([]).to(torch.int64)

# 앙상블 예측 저장 함수 (YOLO 형식: class_id x_center y_center width height confidence)
def save_ensemble_predictions(boxes, scores, classes, output_dir, img_path):
    img_name = os.path.splitext(os.path.basename(img_path))[0]
    output_file = os.path.join(output_dir, f"{img_name}.txt")
    with open(output_file, 'w') as f:  # 항상 파일 생성, 빈 경우도 포함
        if len(boxes) > 0:
            img = cv2.imread(img_path)
            if img is None:
                print(f"Error: Could not load image {img_path}")
                return
            h, w = img.shape[:2]
            for box, score, cls in zip(boxes, scores, classes):
                x1, y1, x2, y2 = box.tolist()
                x_center = (x1 + x2) / 2 / w
                y_center = (y1 + y2) / 2 / h
                width = (x2 - x1) / w
                height = (y2 - y1) / h
                class_id = int(cls.item())
                confidence = float(score.item())
                f.write(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f} {confidence:.6f}\n")

# 예측 시각화 함수
def visualize_predictions(img_path, boxes, scores, classes, class_names, output_dir):
    img = cv2.imread(img_path)
    if img is None:
        print(f"Error: Could not load image {img_path}")
        return
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_name = os.path.splitext(os.path.basename(img_path))[0]
    for box, score, cls in zip(boxes, scores, classes):
        x1, y1, x2, y2 = box.int().tolist()
        class_id = int(cls.item())
        label = f"{class_names[class_id]}: {score:.2f}"
        color = (0, 255, 0)  # Green
        cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
        cv2.putText(img, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    plt.imshow(img)
    plt.axis('off')
    plt.title(f"Ensemble Predictions: {img_name}")
    plt.savefig(os.path.join(output_dir, f"{img_name}.jpg"))
    plt.close()

# 앙상블 예측 함수 (모델 수 유연)
def ensemble_predictions(models, weights, model_names, image_dir, output_dir, conf=0.15, iou=0.4):
    if len(models) != len(weights):
        raise ValueError("Number of models and weights must match.")
    if abs(sum(weights) - 1.0) > 1e-6:
        raise ValueError("Weights must sum to 1.0.")
    if not models:
        raise ValueError("At least one model is required.")
    
    os.makedirs(output_dir, exist_ok=True)
    for img_path in glob.glob(f'{image_dir}/*.jpg'):
        try:
            # 모든 모델로 예측
            all_boxes, all_scores, all_classes = [], [], []
            for model, weight in zip(models, weights):
                results = model.predict(img_path, conf=conf, device='cuda')[0]
                boxes, scores, classes = extract_predictions(results)
                if len(boxes) > 0:
                    scores = scores * weight  # 모델별 가중치 적용
                    all_boxes.append(boxes)
                    all_scores.append(scores)
                    all_classes.append(classes)
            
            # 예측 결합
            all_boxes = torch.cat(all_boxes) if all_boxes else torch.tensor([]).to(torch.float32)
            all_scores = torch.cat(all_scores) if all_scores else torch.tensor([]).to(torch.float32)
            all_classes = torch.cat(all_classes) if all_classes else torch.tensor([]).to(torch.int64)
            
            # 클래스별 NMS
            final_boxes, final_scores, final_classes = [], [], []
            for cls in torch.unique(all_classes):
                cls_mask = all_classes == cls
                cls_boxes = all_boxes[cls_mask]
                cls_scores = all_scores[cls_mask]
                indices = nms(cls_boxes, cls_scores, iou_threshold=iou)
                final_boxes.append(cls_boxes[indices])
                final_scores.append(cls_scores[indices])
                final_classes.append(all_classes[cls_mask][indices])
            
            # 최종 결과 결합
            final_boxes = torch.cat(final_boxes) if final_boxes else torch.tensor([]).to(torch.float32)
            final_scores = torch.cat(final_scores) if final_scores else torch.tensor([]).to(torch.float32)
            final_classes = torch.cat(final_classes) if final_classes else torch.tensor([]).to(torch.int64)
            
            # 결과 저장 및 시각화
            save_ensemble_predictions(final_boxes, final_scores, final_classes, output_dir, img_path)
            # visualize_predictions(img_path, final_boxes, final_scores, final_classes, class_names, output_dir)
            print(f"Processed: {os.path.basename(img_path)}")
        except Exception as e:
            print(f"Error processing {img_path}: {e}")

# F1 스코어 계산 함수
def calculate_f1_score(ground_truth_dir, pred_dir):
    true_positives = 0
    false_positives = 0
    false_negatives = 0
    gt_files = glob.glob(f'{ground_truth_dir}/*.txt')
    pred_files = glob.glob(f'{pred_dir}/*.txt')
    for gt_file in gt_files:
        img_name = os.path.splitext(os.path.basename(gt_file))[0]
        pred_file = os.path.join(pred_dir, f'{img_name}.txt')
        gt_boxes = []
        if os.path.exists(gt_file):
            with open(gt_file, 'r') as f:
                for line in f:
                    parts = list(map(float, line.strip().split()))
                    gt_boxes.append((int(parts[0]), parts[1:]))
        pred_boxes = []
        if os.path.exists(pred_file):
            with open(pred_file, 'r') as f:
                for line in f:
                    parts = list(map(float, line.strip().split()))
                    pred_boxes.append((int(parts[0]), parts[1:5], parts[5]))
        for pred_class, pred_box, conf in pred_boxes:
            matched = False
            for gt_class, gt_box in gt_boxes:
                if pred_class == gt_class:
                    iou = calculate_iou(pred_box, gt_box)
                    if iou > 0.45:
                        true_positives += 1
                        matched = True
                        break
            if not matched:
                false_positives += 1
        false_negatives += len(gt_boxes) - sum(1 for p in pred_boxes if any(p[0] == gt[0] for gt in gt_boxes))
    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    return precision, recall, f1_score

# 클래스별 F1 스코어 계산 함수
def calculate_class_f1_score(ground_truth_dir, pred_dir):
    class_metrics = {name: {'tp': 0, 'fp': 0, 'fn': 0} for name in class_names}
    gt_files = glob.glob(f'{ground_truth_dir}/*.txt')
    for gt_file in gt_files:
        img_name = os.path.splitext(os.path.basename(gt_file))[0]
        pred_file = os.path.join(pred_dir, f'{img_name}.txt')
        gt_boxes = []
        if os.path.exists(gt_file):
            with open(gt_file, 'r') as f:
                for line in f:
                    parts = list(map(float, line.strip().split()))
                    gt_boxes.append((int(parts[0]), parts[1:]))
        pred_boxes = []
        if os.path.exists(pred_file):
            with open(pred_file, 'r') as f:
                for line in f:
                    parts = list(map(float, line.strip().split()))
                    pred_boxes.append((int(parts[0]), parts[1:5], parts[5]))
        for pred_class, pred_box, conf in pred_boxes:
            matched = False
            for gt_class, gt_box in gt_boxes:
                if pred_class == gt_class:
                    iou = calculate_iou(pred_box, gt_box)
                    if iou > 0.45:
                        class_metrics[class_names[pred_class]]['tp'] += 1
                        matched = True
                        break
            if not matched:
                class_metrics[class_names[pred_class]]['fp'] += 1
        for gt_class, gt_box in gt_boxes:
            if not any(pred_class == gt_class for pred_class, _, _ in pred_boxes):
                class_metrics[class_names[gt_class]]['fn'] += 1
    for name in class_names:
        metrics = class_metrics[name]
        precision = metrics['tp'] / (metrics['tp'] + metrics['fp']) if (metrics['tp'] + metrics['fp']) > 0 else 0
        recall = metrics['tp'] / (metrics['tp'] + metrics['fn']) if (metrics['tp'] + metrics['fn']) > 0 else 0
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        print(f"{name} - Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")
    return class_metrics

# IoU 계산 함수
def calculate_iou(box1, box2):
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2
    x1_min, y1_min = x1 - w1/2, y1 - h1/2
    x1_max, y1_max = x1 + w1/2, y1 + h1/2
    x2_min, y2_min = x2 - w2/2, y2 - h2/2
    x2_max, y2_max = x2 + w2/2, y2 + h2/2
    inter_x_min = max(x1_min, x2_min)
    inter_y_min = max(y1_min, y2_min)
    inter_x_max = min(x1_max, x2_max)
    inter_y_max = min(y1_max, y2_max)
    inter_area = max(0, inter_x_max - inter_x_min) * max(0, inter_y_max - inter_y_min)
    box1_area = w1 * h1
    box2_area = w2 * h2
    union_area = box1_area + box2_area - inter_area
    return inter_area / union_area if union_area > 0 else 0

# 모든 조합에 대해 앙상블 실행
print("Starting ensemble predictions for all combinations...")
combinations_list = [
    ([0, 1], [0.5, 0.5], "YOLOv8xl_YOLOv11l"),  # (model1, model2)
    ([0, 2], [0.5, 0.5], "YOLOv8xl_YOLOv11xl"),  # (model1, model3)
    ([1, 2], [0.5, 0.5], "YOLOv11l_YOLOv11xl"),  # (model2, model3)
    ([0, 1, 2], [0.3, 0.3, 0.4], "YOLOv8xl_YOLOv11l_YOLOv11xl")  # (model1, model2, model3)
]

for model_indices, weights, combo_name in combinations_list:
    print(f"\nRunning ensemble for {combo_name}...")
    selected_models = [all_models[i] for i in model_indices]
    selected_names = [model_names[i] for i in model_indices]
    output_dir = os.path.join(base_output_dir, f'ensemble_{combo_name}')
    ensemble_predictions(selected_models, weights, selected_names, test_image_dir, output_dir, conf=0.3, iou=0.4)
    print(f"Predictions saved to {output_dir}")
    
    # F1 스코어 계산
    precision, recall, f1_score = calculate_f1_score(ground_truth_dir, output_dir)
    print(f"{combo_name} - Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1_score:.4f}")
    

###**Conclusion**
In this tutorial, you learned how to:

1.  Install and set up YOLOv8 using CLI commands.

2.  Download a pre-trained YOLOv8 model.

3.  Prepare a custom dataset and configure the data.yaml file.

4.  Train the YOLOv8 model on your dataset.

5.  Evaluate the model and test it with new images, videos, or real-time webcam feeds.

6.  Save and load the trained model for future use.